In [7]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import xarray as xr

import requests
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
import os
import shutil
import time

import concurrent.futures
import glob

In [8]:
places_coordinates = {
    "Europe": {'lat_min': 35.25, 'lat_max': 71.75, 'lon_min': -10.25, 'lon_max': 31},
    "Canada": {'lat_min': 44.0, 'lat_max': 70.0, 'lon_min': -141.5, 'lon_max': -51.5}, 
    "US": {'lat_min': 24, 'lat_max': 49, 'lon_min': -125.5, 'lon_max': -65.5},
    "BlackSea": {'lat_min': 41.25, 'lat_max': 57.0, 'lon_min':22.75, 'lon_max': 61.5},
    "Australia": {'lat_min': -39, 'lat_max': -10.5, 'lon_min': 112.75, 'lon_max': 153.25},
    "Argentina": {'lat_min': -56.25, 'lat_max': -21.75, 'lon_min': -75.5, 'lon_max': -51.75}
}

Before 2021

In [3]:
def find_dates(year, max_retries=5, start_date=False):
    retries = 0
    while retries < max_retries:
        try:
            r = requests.get(f'https://tds.gdex.ucar.edu/thredds/catalog/files/g/d084001/{year}/catalog.html', timeout=10)
            break
        except requests.exceptions.RequestException as e:
            retries += 1
            time.sleep(5)
            
    soup = BeautifulSoup(r.text, "html.parser")
    dates = [f.get_text() for f in soup.find_all('code') if re.fullmatch(r"\d{8}", f.get_text())]
    if start_date:
        return [d for d in dates if datetime.strptime(d, '%Y%m%d') >= datetime.strptime(str(start_date), '%Y%m%d')]
    else:
        return dates

In [4]:
def find_files(year, date, run, variable, forecast_horizon, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            r = requests.get(f'https://tds.gdex.ucar.edu/thredds/catalog/files/g/d084001/{year}/{date}/catalog.html')
            break
        except requests.exceptions.RequestException as e:
            retries += 1
            time.sleep(5)
            
    soup = BeautifulSoup(r.text, "html.parser")
    files =  [f.get_text() for f in soup.find_all('code') if f.get_text()[-6:] == '.grib2']
    current_run = [f for f in files if f.split('.')[2][-2:] == run  # check if run is 00
                   and int(f.split('.f')[1][:3]) > 0 #remove the first data, where data is initial values of the model and not forecast
                   and int(f.split('.f')[1][:3]) <= forecast_horizon] #where files are max forecast horizon
    precip_run = [f for f in current_run if int(f.split('.f')[1][:3]) > 0 #remove the first data, where data is initial values of the model and not forecast
                  and int(f.split('.f')[1][:3]) % 6 == 0 #every 6 hours
                  and int(f.split('.f')[1][:3]) <= forecast_horizon] #where files are max forecast horizon

    match variable:
        case 'Temp':
            return current_run
        case 'Precip':
            return precip_run
        case _:
            raise ValueError('Variable pas connue')

In [5]:
def download_single_file(year, date, file, coords, variable, filepath, max_retries=5):
    forecast_hour = int(file.split('.f')[1][:3])
    forecast_hour_timedelta = timedelta(hours=forecast_hour)
    ref_date =  datetime.strptime(file.split('.')[2], "%Y%m%d%H")
    final_dt = ref_date + forecast_hour_timedelta
    iso_date = final_dt.strftime("%Y-%m-%dT%H:%M:%SZ")        
            
    match variable:
        case 'Temp':
            var = 'var=Temperature_height_above_ground'
        case 'Precip':
            if ref_date < datetime(2019, 6, 13):
                var = 'var=Total_precipitation_surface_6_Hour_Accumulation'
            elif forecast_hour == 6: 
                var = 'var=Total_precipitation_surface_6_Hour_Accumulation'
            else:
                var = 'var=Total_precipitation_surface_Mixed_intervals_Accumulation'

    endpoint = "https://tds.gdex.ucar.edu/thredds/ncss/grid/files/g/d084001/"
    url_params = f"{year}/{date}/{file}?{var}&" \
                        f"north={coords['lat_max']}&west={coords['lon_min']}&east={coords['lon_max']}&south={coords['lat_min']}&" \
                        f"horizStride=1&time_start={iso_date}&time_end={iso_date}&&&accept=netcdf3"
    retries = 0
    while retries < max_retries:
        try:
            r = requests.get(endpoint+url_params, timeout=10)
            r.raise_for_status()
            with open(f'{filepath}/gfs.0p25.{iso_date.replace(":", "-")}.grib2.nc', "wb") as f:
                f.write(r.content)
            break
        except requests.exceptions.RequestException as e:
            retries += 1
            time.sleep(1)
    else:
        print("Max retries reached. Download failed for ", file)

def download_wrapper(file):
    """Wrapper pour gérer les retries et exceptions par fichier."""
    try:
        download_single_file(year, date, file, coords, variable, f"{variable}/{place}/{year}/{date}")
        return True
    except Exception as e:
        print(f"Download failed for {file}: {e}")
        return False

In [6]:
def process_files(year, date, place, variable):
    def fix_coords(ds):
        # Harmoniser la coordonnée temporelle
        if "time1" in ds.coords:      # si le fichier contient "time1"
            ds = ds.rename({"time1": "time"})
        if "time2" in ds.coords:
            ds = ds.rename({"time2": "time"})
        if "height_above_ground1" in ds.coords:
            ds = ds.rename({"height_above_ground1": "height_above_ground"})
        if "height_above_ground2" in ds.coords:
            ds = ds.rename({"height_above_ground2": "height_above_ground"})
        if "height_above_ground3" in ds.coords:
            ds = ds.rename({"height_above_ground3": "height_above_ground"})
        # Same for precip variable name
        if "Total_precipitation_surface_6_Hour_Accumulation" in ds.data_vars:
            ds = ds.rename_vars({"Total_precipitation_surface_6_Hour_Accumulation": "Total_precipitation_surface_Mixed_intervals_Accumulation"})
        return ds
    
    ds = xr.open_mfdataset(f'{variable}/{place}/{year}/{date}/*.nc', engine='netcdf4', preprocess=fix_coords, decode_timedelta=True)

    if ds.longitude.max() > 180:
        ds = ds.assign_coords(longitude=((ds.longitude + 180) % 360) - 180)
        ds = ds.sortby("longitude")

    match variable:
        case 'Temp':
            ds_2m = ds.sel(height_above_ground=2)
            t2m = ds_2m['Temperature_height_above_ground'].mean(dim='time')
            df_t2m = t2m.to_dataframe().reset_index()
            df_t2m['Temperature_height_above_ground'] = df_t2m['Temperature_height_above_ground'] - 273.15
            df_t2m = df_t2m[['reftime', 'Temperature_height_above_ground', 'latitude', 'longitude']]
            if not df_t2m.empty:
                df_t2m.to_parquet(f"{variable}/{place}/{year}/{date}_processed.parquet", compression='snappy')
                #shutil.rmtree(f"{variable}/{place}/{year}/{date}")
            else:
                raise ValueError(f"{variable}, {date}, df is empty")
        case 'Precip':
            precip = ds['Total_precipitation_surface_Mixed_intervals_Accumulation'].sum(dim='time')
            df_precip = precip.to_dataframe().reset_index()
            df_precip = df_precip[['reftime', 'Total_precipitation_surface_Mixed_intervals_Accumulation', 'latitude', 'longitude']]
            if not df_precip.empty:
                df_precip.to_parquet(f"{variable}/{place}/{year}/{date}_processed.parquet", compression='snappy')
                #shutil.rmtree(f"{variable}/{place}/{year}/{date}")
            else:
                raise ValueError(f"{variable}, {date}, df is empty")
        case _:
            raise ValueError("Variable needs to be 'Temp' or 'Precip'")

In [ ]:
years = [i for i in range(2015, 2022)]
for variable in ['Temp', 'Precip']:   
    if variable == 'Temp': #Don't do one of them
        continue
    os.makedirs(f"{variable}", exist_ok=True)
    for place, coords in places_coordinates.items():
        if place == "Europe" or place == "Canada": # Move to next place if needed
            continue
        os.makedirs(f"{variable}/{place}", exist_ok=True)
        for year in years:
            os.makedirs(f"{variable}/{place}/{year}", exist_ok=True)
            dates = find_dates(year)
            for date in dates:
                os.makedirs(f"{variable}/{place}/{year}/{date}", exist_ok=True)
                files = find_files(year, date, '00', variable, 168)
                # Télécharger en parallèle
                with concurrent.futures.ThreadPoolExecutor(max_workers=14) as executor:  # Ajuste le nombre de threads
                    results = list(executor.map(download_wrapper, files))

                # Vérifier que tout s'est bien passé
                if all(results):
                    process_files(year, date, place, variable)
                else:
                    failed_files = [file for file, success in zip(files, results) if not success]
                    print("Downloads failed for files:", failed_files)

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_parquet('Temp/Argentina/2015/20150115_processed.parquet')

fig = px.scatter_geo(
    df,
    lat="latitude",
    lon="longitude",
    color="Temperature_height_above_ground",
    labels={"Temperature_height_above_ground": "Temp"},
    color_continuous_scale=px.colors.sequential.Viridis
)
fig.update_geos(fitbounds="locations")
fig.update_layout(title="Temperature_height_above_ground", title_x=0.5)
fig.show()

After 2021

In [9]:
GFS_BUCKET_NAME = "noaa-gfs-bdp-pds"
FORECAST_CYCLE = "00"
FORECAST_MODEL = "atmos"
FILE_TYPE = "pgrb2"
GRID_RESOLUTION = "0p25" 

TEMP_KEYS = {'filter_by_keys': {'typeOfLevel': 'heightAboveGround', 'level': 2, 'cfVarName': 't2m'}, 'indexpath': ''}
PRECIP_KEYS = {'filter_by_keys': {'typeOfLevel': 'surface', 'cfVarName': 'tp'}, 'indexpath': ''}

In [10]:
def get_files(s3, date):    
    folder = f"gfs.{date}/{FORECAST_CYCLE}/{FORECAST_MODEL}/"
    try:
        response = s3.list_objects_v2(Bucket=GFS_BUCKET_NAME, Prefix=folder, Delimiter='/')
        if not 'Contents' in response.keys():
            folder = f"gfs.{date}/{FORECAST_CYCLE}/"
            response = s3.list_objects_v2(Bucket=GFS_BUCKET_NAME, Prefix=folder, Delimiter='/')
    except Exception as e:
        print(f"Error accessing S3: {e}")

    pgrb2_files = [obj['Key'] for obj in response['Contents'] if f"gfs.t{FORECAST_CYCLE}z.{FILE_TYPE}.{GRID_RESOLUTION}.f" in obj['Key']]
    file_res = [file for file in pgrb2_files if file[-4:] != ".idx"]
    file_horizon = [file for file in file_res if int(file.split('.')[5][-3:]) <= 168
                    and int(file.split('.')[5][-3:]) > 0
                    and int(file.split('.')[5][-3:]) % 3 == 0]
                    
    return file_horizon

In [11]:
def download_files(s3, files, filepath):
    for file in files:
        try:
            filename = file.split('/')[-1]
            os.makedirs(filepath, exist_ok=True)
            s3.download_file(GFS_BUCKET_NAME, file, f'{filepath}/{filename}')
        except Exception as e:
            print(f"Error downloading {file}: {e}")

In [12]:
def process_files(year, date, places):
    files = sorted(glob.glob(f'{year}/{date}/*'))  

    acc = None
    n = 0

    for f in files:
        #ds = xr.open_dataset(f, engine='cfgrib', backend_kwargs=kwargs, decode_timedelta=True)
        ds_t2m = xr.open_dataset(f, engine="cfgrib", backend_kwargs=TEMP_KEYS, decode_timedelta=True)
        ds_tp = xr.open_dataset(f, engine="cfgrib", backend_kwargs=PRECIP_KEYS, decode_timedelta=True)
        ds = xr.merge([ds_t2m, ds_tp])
        
        if ds.longitude.max() > 180:
            ds = ds.assign_coords(longitude=((ds.longitude + 180) % 360) - 180)
            ds = ds.sortby("longitude")

        if acc is None:
            acc = ds.copy(deep=False)
        else:
            acc += ds

        n += 1

    t2m_mean = acc['t2m'] / n
    t2m_mean -= 273.15
    precip_sum = acc['tp']

    for place, coords in places.items():
        sl = dict(
            latitude=slice(coords['lat_max'], coords['lat_min']),
            longitude=slice(coords['lon_min'], coords['lon_max'])
        )
        t2m_sliced = t2m_mean.sel(**sl)
        precip_sliced = precip_sum.sel(**sl)
    
        df_t2m = t2m_sliced.to_dataframe().reset_index()
        df_t2m = df_t2m[['time', 'latitude', 'longitude', 't2m']]

        df_precip = precip_sliced.to_dataframe().reset_index()
        df_precip = df_precip[['time', 'latitude', 'longitude', 'tp']]
        
        if df_t2m.empty or df_precip.empty:
            raise ValueError(f"{variable}, {date}, one of the 2 df is empty")

        df_t2m.to_parquet(f"Temp/{place}/{year}/{date}_processed.parquet", compression="snappy")
        df_precip.to_parquet(f"Precip/{place}/{year}/{date}_processed.parquet", compression="snappy")

In [13]:
def get_dates(s3):
    paginator = s3.get_paginator("list_objects_v2")
    pattern = re.compile(r"gfs\.\d{8}/$")
    gfs_dates = []

    for page in paginator.paginate(Bucket=GFS_BUCKET_NAME, Delimiter="/"):
        for cp in page.get("CommonPrefixes", []):
            prefix = cp["Prefix"]
            if pattern.match(prefix):
                file_name = prefix.rstrip("/")
                gfs_dates.append(file_name.split('.')[1])
    return gfs_dates

In [14]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
dates = get_dates(s3)
for date in dates:
    files = get_files(s3, date)
    year = files[0].split('/')[0][4:8]
    download_files(s3, files, f'{year}/{date}') 
    process_files(year, date, places_coordinates)
    break

c:\Users\alexl\miniconda3\envs\conda-env\Lib\site-packages\xarray\backends\plugins.py:110: RuntimeWarning: Engine 'rasterio' loading failed:
DLL load failed while importing _io: Le module spécifié est introuvable.
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


In [ ]:
#parralel download of file 